[Рекуррентные нейронные сети](https://neerc.ifmo.ru/wiki/index.php?title=%D0%A0%D0%B5%D0%BA%D1%83%D1%80%D1%80%D0%B5%D0%BD%D1%82%D0%BD%D1%8B%D0%B5_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D1%8B%D0%B5_%D1%81%D0%B5%D1%82%D0%B8)

**keras**

In [1]:
# Импорты
import numpy as np
from keras.preprocessing import sequence
from keras.utils.data_utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Dropout
from keras.layers import LSTM
from keras.datasets import imdb

# Устанавливаем seed для обеспечения повторяемости результатов
np.random.seed(42)

In [2]:
# Указываем количество слов из частотного словаря, которое будет использоваться (отсортированы по частоте использования)
max_features = 5000

In [3]:
# Загружаем данные (датасет IMDB содержит 25000 рецензий на фильмы с правильным ответом для обучения и 25000 рецензий на фильмы с правильным ответом для тестирования)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = max_features)

In [4]:
# Устанавливаем максимальную длину рецензий в словах, чтобы они все были одной длины
maxlen = 80

# Заполняем короткие рецензии пробелами, а длинные обрезаем
X_train = pad_sequences(X_train, maxlen = maxlen)
X_test = pad_sequences(X_test, maxlen = maxlen)

In [5]:
# Создаем модель последовательной сети
model = Sequential()

# Добавляем слой для векторного представления слов (5000 слов, каждое представлено вектором из 32 чисел, отключаем входной сигнал с вероятностью 20% для предотвращения переобучения)
model.add(Embedding(max_features, 32))
model.add(Dropout(0.2))
# Добавляем слой долго-краткосрочной памяти (100 элементов для долговременного хранения информации, отключаем входной сигнал с вероятностью 20%, отключаем рекуррентный сигнал с вероятностью 20%)
model.add(LSTM(100))
model.add(Dropout(0.2))
# Добавляем полносвязный слой из 1 элемента для классификации, в качестве функции активации будем использовать сигмоидальную функцию
model.add(Dense(1, activation = 'sigmoid'))

# Компилируем модель нейронной сети
model.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          160000    
                                                                 
 dropout (Dropout)           (None, None, 32)          0         
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Обучаем нейронную сеть (данные для обучения, ответы к данным для обучения, количество рецензий после анализа которого будут изменены веса, число эпох обучения, тестовые данные, показывать progress bar или нет)
model.fit(
    X_train, 
    y_train, 
    batch_size = 64,
    epochs = 7,
    validation_data = (X_test, y_test),
    verbose = 1
)

Epoch 1/7
391/391 [==============================] - 34s 84ms/step - loss: 0.4573 - accuracy: 0.7729 - val_loss: 0.3621 - val_accuracy: 0.8398
Epoch 2/7
391/391 [==============================] - 36s 92ms/step - loss: 0.3206 - accuracy: 0.8635 - val_loss: 0.3637 - val_accuracy: 0.8405
Epoch 3/7
391/391 [==============================] - 36s 91ms/step - loss: 0.2907 - accuracy: 0.8802 - val_loss: 0.3607 - val_accuracy: 0.8404
Epoch 4/7
391/391 [==============================] - 35s 90ms/step - loss: 0.2578 - accuracy: 0.8950 - val_loss: 0.3745 - val_accuracy: 0.8355
Epoch 5/7
391/391 [==============================] - 35s 89ms/step - loss: 0.2313 - accuracy: 0.9048 - val_loss: 0.4103 - val_accuracy: 0.8300
Epoch 6/7
391/391 [==============================] - 35s 90ms/step - loss: 0.2066 - accuracy: 0.9155 - val_loss: 0.4686 - val_accuracy: 0.8218
Epoch 7/7
391/391 [==============================] - 34s 87ms/step - loss: 0.1862 - accuracy: 0.9236 - val_loss: 0.4720 - val_accuracy: 0.8249

In [9]:
# Проверяем качество обучения на тестовых данных (если есть данные, которые не участвовали в обучении, лучше использовать их, но в нашем случае таковых нет)
scores = model.evaluate(X_test, y_test, batch_size = 64)

print('Точность на тестовых данных: %.2f%%' % (scores[1] * 100))

391/391 [==============================] - 8s 20ms/step - loss: 0.4720 - accuracy: 0.8249
Точность на тестовых данных: 82.49%
